# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
## import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import TruncatedSVD
import pickle
import re

In [2]:
# load data from database
engine = create_engine('sqlite:///mydb.db')
df = pd.read_sql_table('mytable',engine)
X = df["message"]
Y = df.drop(['message', 'genre', 'id', 'original'], axis = 1)


In [3]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [4]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

In [5]:
def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
import nltk
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /Users/mac/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/mac/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size = 0.2, random_state = 45)
# train classifier
pipeline.fit(X_train, y_train)

/Users/mac/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/mac/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/mac/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/mac/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/mac/anaconda3/lib/python3.6/site-packages/skl

/Users/mac/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                  

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
def report(model,X_test,y_test):
    
    
    y_pred = pipeline.predict(X_test)
    
    for i,col in enumerate(y_test):
        
        print(classification_report(y_test[col], y_pred[:, i]))
        
        
        
    

In [10]:
report(pipeline, X_test, y_test)

              precision    recall  f1-score   support

           0       0.59      0.37      0.45      1198
           1       0.82      0.93      0.87      4002
           2       0.67      0.14      0.23        44

    accuracy                           0.79      5244
   macro avg       0.70      0.48      0.52      5244
weighted avg       0.77      0.79      0.77      5244

              precision    recall  f1-score   support

           0       0.88      0.99      0.93      4335
           1       0.84      0.38      0.52       909

    accuracy                           0.88      5244
   macro avg       0.86      0.68      0.73      5244
weighted avg       0.88      0.88      0.86      5244

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5214
           1       0.00      0.00      0.00        30

    accuracy                           0.99      5244
   macro avg       0.50      0.50      0.50      5244
weighted avg       0

/Users/mac/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5195
           1       1.00      0.04      0.08        49

    accuracy                           0.99      5244
   macro avg       1.00      0.52      0.54      5244
weighted avg       0.99      0.99      0.99      5244

              precision    recall  f1-score   support

           0       0.96      0.99      0.98      4762
           1       0.90      0.64      0.75       482

    accuracy                           0.96      5244
   macro avg       0.93      0.81      0.86      5244
weighted avg       0.96      0.96      0.96      5244

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      5136
           1       0.92      0.10      0.18       108

    accuracy                           0.98      5244
   macro avg       0.95      0.55      0.59      5244
weighted avg       0.98      0.98      0.97      5244

              preci

### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
parameters =  {'tfidf__use_idf': (True, False), 
              'clf__estimator__n_estimators': [50, 100], 
              'clf__estimator__min_samples_split': [2, 4]} 

cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [12]:
cv.fit(X_train, y_train)


/Users/mac/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                       

In [13]:
report(cv, X_test, y_test)

              precision    recall  f1-score   support

           0       0.59      0.37      0.45      1198
           1       0.82      0.93      0.87      4002
           2       0.67      0.14      0.23        44

    accuracy                           0.79      5244
   macro avg       0.70      0.48      0.52      5244
weighted avg       0.77      0.79      0.77      5244

              precision    recall  f1-score   support

           0       0.88      0.99      0.93      4335
           1       0.84      0.38      0.52       909

    accuracy                           0.88      5244
   macro avg       0.86      0.68      0.73      5244
weighted avg       0.88      0.88      0.86      5244

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5214
           1       0.00      0.00      0.00        30

    accuracy                           0.99      5244
   macro avg       0.50      0.50      0.50      5244
weighted avg       0

/Users/mac/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5195
           1       1.00      0.04      0.08        49

    accuracy                           0.99      5244
   macro avg       1.00      0.52      0.54      5244
weighted avg       0.99      0.99      0.99      5244

              precision    recall  f1-score   support

           0       0.96      0.99      0.98      4762
           1       0.90      0.64      0.75       482

    accuracy                           0.96      5244
   macro avg       0.93      0.81      0.86      5244
weighted avg       0.96      0.96      0.96      5244

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      5136
           1       0.92      0.10      0.18       108

    accuracy                           0.98      5244
   macro avg       0.95      0.55      0.59      5244
weighted avg       0.98      0.98      0.97      5244

              preci

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [14]:
#Improve  the pipeline
pipeline2 = Pipeline([
    ('vect', CountVectorizer()),
    ('best', TruncatedSVD()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

In [15]:
#Train & predict
pipeline2.fit(X_train, y_train)
report(pipeline2, X_test, y_test)

              precision    recall  f1-score   support

           0       0.59      0.37      0.45      1198
           1       0.82      0.93      0.87      4002
           2       0.67      0.14      0.23        44

    accuracy                           0.79      5244
   macro avg       0.70      0.48      0.52      5244
weighted avg       0.77      0.79      0.77      5244

              precision    recall  f1-score   support

           0       0.88      0.99      0.93      4335
           1       0.84      0.38      0.52       909

    accuracy                           0.88      5244
   macro avg       0.86      0.68      0.73      5244
weighted avg       0.88      0.88      0.86      5244

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5214
           1       0.00      0.00      0.00        30

    accuracy                           0.99      5244
   macro avg       0.50      0.50      0.50      5244
weighted avg       0

/Users/mac/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5195
           1       1.00      0.04      0.08        49

    accuracy                           0.99      5244
   macro avg       1.00      0.52      0.54      5244
weighted avg       0.99      0.99      0.99      5244

              precision    recall  f1-score   support

           0       0.96      0.99      0.98      4762
           1       0.90      0.64      0.75       482

    accuracy                           0.96      5244
   macro avg       0.93      0.81      0.86      5244
weighted avg       0.96      0.96      0.96      5244

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      5136
           1       0.92      0.10      0.18       108

    accuracy                           0.98      5244
   macro avg       0.95      0.55      0.59      5244
weighted avg       0.98      0.98      0.97      5244

              preci

In [16]:
#Param tunning 
parameters2 = {
              'tfidf__use_idf': (True, False), 
              'clf__estimator__n_estimators': [50, 100],
              'clf__estimator__learning_rate': [1,2] }

In [17]:
cv2 = GridSearchCV(pipeline2, param_grid=parameters2)
cv2

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                       

In [18]:

cv2.fit(X_train, y_train)

/Users/mac/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                       

### 9. Export your model as a pickle file

In [19]:
with open('model.pkl', 'wb') as f:
    pickle.dump(cv2, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.